In [1]:
import numpy as np
from glob import glob
import matplotlib.image as mpimg
from numpy.linalg import svd, qr, norm,  inv
from All import unfold, fold, Mul_1, Mul_2, Mul_3
from sklearn.metrics import classification_report

In [2]:
list_of_name_of_images = glob("YALE faces/yalefaces/train/*.gif")
list_of_expression = [list_of_name_of_images[i].split(".")[1] for i in range(9)]

In [3]:
list_of_expression

['centerlight',
 'happy',
 'leftlight',
 'normal',
 'rightlight',
 'sad',
 'sleepy',
 'surprised',
 'wink']

In [4]:
array_of_list_of_persons = [[] for i in range(15)]
slice = range(0, 136, 9)
A = np.empty((77760, 9, 15))

for i in range(15):

    for filename in list_of_name_of_images[slice[i] : slice[i+1]]:
        
        image = mpimg.imread(filename)
        array_of_list_of_persons[i].append(image.flatten(order="f"))
        
    A[:, :, i] = np.matrix(array_of_list_of_persons[i]).T

In [5]:
mpimg.imread(list_of_name_of_images[0]).shape

(243, 320)

# unfolds

In [6]:
unfold_1 = unfold(A, 1)
unfold_2 = unfold(A, 2)
unfold_3 = unfold(A, 3)

# compute F, G, H, S, B

In [7]:
F, _, _ = svd(unfold_1, full_matrices=False)
G, _, _ = svd(unfold_2, full_matrices=False)
H, _, _ = svd(unfold_3, full_matrices=False)

temp_1 = Mul_1(F.T, A)
temp_2 = Mul_2(G.T, temp_1)
S = Mul_3(H.T, temp_2)

B = Mul_2(G, S)

# Creating list of Test images

In [8]:
list_of_test_images = glob("YALE faces/yalefaces/test/*.gif")
array_list_of_test_images = [mpimg.imread(filename).flatten(order="f") for filename in list_of_test_images]
len(array_list_of_test_images)

30

In [9]:
true_label = []
for i in range(1, 16):
    for j in range(2):
        true_label.append(i)

In [10]:
true_label

[1,
 1,
 2,
 2,
 3,
 3,
 4,
 4,
 5,
 5,
 6,
 6,
 7,
 7,
 8,
 8,
 9,
 9,
 10,
 10,
 11,
 11,
 12,
 12,
 13,
 13,
 14,
 14,
 15,
 15]

# Compute All Be = Qe * Re

In [11]:
n_e, _ = G.shape
n_p, _ = H.shape
list_of_Qe_Re = []

for expression in range(n_e):
    Q_e, R_e = qr(B[:, expression, :], mode="reduced")
    list_of_Qe_Re.append([Q_e, R_e])

# Face Recognition

In [12]:
def face_recognition(test_image, list_of_Qe_Re, list_of_prediction, tol):

    z_hat = np.dot(F.T, test_image)

    for Q_e, R_e in list_of_Qe_Re:

        a_e = np.dot(np.dot(inv(R_e), Q_e.T), z_hat)

        for person in range(n_p):

            if norm(a_e - H[person, :], 2) < tol:

                list_of_prediction.append(person+1)
                return

In [13]:
def face_recognition_without_tol(test_image, list_of_Qe_Re, list_of_prediction):

    z_hat = np.dot(F.T, test_image)

    list_of_distance = []

    for Q_e, R_e in list_of_Qe_Re:

        a_e = np.dot(np.dot(inv(R_e), Q_e.T), z_hat)

        for person in range(n_p):

            list_of_distance.append((norm(a_e - H[person, :], 2), person + 1))

    list_of_distance = sorted(list_of_distance)
    list_of_prediction.append(list_of_distance[0][1])
        

# Accuarcy of function with hyperparameter tol

In [22]:
list_of_prediction = []
for test_image in array_list_of_test_images:

    face_recognition(test_image, list_of_Qe_Re, list_of_prediction, 0.75)

In [23]:
accuracy = (np.count_nonzero((np.array(list_of_prediction) - np.array(true_label)) == 0) / 30) * 100
print(f"accuracy of Face Recognition is = {accuracy:.2f}")

accuracy of Face Recognition is = 80.00


In [16]:
print(classification_report(np.array(true_label), np.array(list_of_prediction)))

              precision    recall  f1-score   support

           1       1.00      0.50      0.67         2
           2       0.67      1.00      0.80         2
           3       1.00      1.00      1.00         2
           4       1.00      1.00      1.00         2
           5       0.67      1.00      0.80         2
           6       0.00      0.00      0.00         2
           7       0.50      1.00      0.67         2
           8       1.00      1.00      1.00         2
           9       1.00      1.00      1.00         2
          10       1.00      0.50      0.67         2
          11       1.00      1.00      1.00         2
          12       1.00      0.50      0.67         2
          13       1.00      1.00      1.00         2
          14       0.50      1.00      0.67         2
          15       1.00      0.50      0.67         2

    accuracy                           0.80        30
   macro avg       0.82      0.80      0.77        30
weighted avg       0.82   

c:\Users\mahan\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\mahan\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\mahan\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# Accuarcy of function without hyperparameter tol

In [17]:
list_of_prediction = []
for test_image in array_list_of_test_images:

    face_recognition_without_tol(test_image, list_of_Qe_Re, list_of_prediction)

In [20]:
accuracy = (np.count_nonzero((np.array(list_of_prediction) - np.array(true_label)) == 0) / 30) * 100
print(f"accuracy of Face Recognition is = {accuracy:.2f}")

accuracy of Face Recognition is = 93.33


In [19]:
print(classification_report(np.array(true_label), np.array(list_of_prediction)))

              precision    recall  f1-score   support

           1       1.00      0.50      0.67         2
           2       0.67      1.00      0.80         2
           3       1.00      1.00      1.00         2
           4       1.00      1.00      1.00         2
           5       1.00      1.00      1.00         2
           6       1.00      1.00      1.00         2
           7       1.00      1.00      1.00         2
           8       0.67      1.00      0.80         2
           9       1.00      1.00      1.00         2
          10       1.00      0.50      0.67         2
          11       1.00      1.00      1.00         2
          12       1.00      1.00      1.00         2
          13       1.00      1.00      1.00         2
          14       1.00      1.00      1.00         2
          15       1.00      1.00      1.00         2

    accuracy                           0.93        30
   macro avg       0.96      0.93      0.93        30
weighted avg       0.96   